# Final Project - Pre-Processing

In [1]:
import re
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import types, Row, Column
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [3]:
# read in raw text set and write to parquet
# train = spark.read.option('header', 'false').csv('data/train.txt', sep='\t')
# train.write.format('parquet').save('data/train.parquet')

In [3]:
# read in parqet
train = spark.read.parquet('data/train.parquet')

In [4]:
# rename label column
train = train.withColumnRenamed('_c0', 'label')

# remove underscores
for c in train.columns[1:]:
    train = train.withColumnRenamed(c, c.strip('_'))

for i,c in enumerate(train.columns[1:14]):
    newName = 'n' + str(i)
    train = train.withColumnRenamed(c, newName)
    
for i,c in enumerate(train.columns[14:]):
    newName = 'c' + str(i)
    train = train.withColumnRenamed(c, newName)

train.columns

['label',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'c0',
 'c1',
 'c2',
 'c3',
 'c4',
 'c5',
 'c6',
 'c7',
 'c8',
 'c9',
 'c10',
 'c11',
 'c12',
 'c13',
 'c14',
 'c15',
 'c16',
 'c17',
 'c18',
 'c19',
 'c20',
 'c21',
 'c22',
 'c23',
 'c24',
 'c25']

In [5]:
# cast numerical is float
for c in train.columns[:14]:
    train = train.withColumn(c, train[c].cast('float'))
train.printSchema()

root
 |-- label: float (nullable = true)
 |-- n0: float (nullable = true)
 |-- n1: float (nullable = true)
 |-- n2: float (nullable = true)
 |-- n3: float (nullable = true)
 |-- n4: float (nullable = true)
 |-- n5: float (nullable = true)
 |-- n6: float (nullable = true)
 |-- n7: float (nullable = true)
 |-- n8: float (nullable = true)
 |-- n9: float (nullable = true)
 |-- n10: float (nullable = true)
 |-- n11: float (nullable = true)
 |-- n12: float (nullable = true)
 |-- c0: string (nullable = true)
 |-- c1: string (nullable = true)
 |-- c2: string (nullable = true)
 |-- c3: string (nullable = true)
 |-- c4: string (nullable = true)
 |-- c5: string (nullable = true)
 |-- c6: string (nullable = true)
 |-- c7: string (nullable = true)
 |-- c8: string (nullable = true)
 |-- c9: string (nullable = true)
 |-- c10: string (nullable = true)
 |-- c11: string (nullable = true)
 |-- c12: string (nullable = true)
 |-- c13: string (nullable = true)
 |-- c14: string (nullable = true)
 |-- c15: st

In [7]:
# grab a sample
s = train.sample(False, 0.00025)
s.count()

11536

In [8]:
s.head()

Row(label=0.0, n0=0.0, n1=0.0, n2=4.0, n3=2.0, n4=3096.0, n5=55.0, n6=1.0, n7=5.0, n8=40.0, n9=0.0, n10=1.0, n11=None, n12=2.0, c0='05db9164', c1='6e638bbc', c2='e3a92241', c3='c771bf5c', c4='25c83c98', c5=None, c6='d7ea84dc', c7='0b153874', c8='a73ee510', c9='cd8f34fb', c10='4a77ddca', c11='4f7b022c', c12='dc1d72e4', c13='051219e6', c14='3d5d2969', c15='c2807520', c16='e5ba7672', c17='3cb7e3f0', c18='21ddcdc9', c19='a458ea53', c20='59a15e58', c21=None, c22='32c7478e', c23='8d653a3e', c24='445bbe3b', c25='8e1ae331')

In [10]:
for c in s.columns[4:14]:
    s = s.drop(c)
s.columns

['label',
 'n0',
 'n1',
 'n2',
 'c0',
 'c1',
 'c2',
 'c3',
 'c4',
 'c5',
 'c6',
 'c7',
 'c8',
 'c9',
 'c10',
 'c11',
 'c12',
 'c13',
 'c14',
 'c15',
 'c16',
 'c17',
 'c18',
 'c19',
 'c20',
 'c21',
 'c22',
 'c23',
 'c24',
 'c25']

In [13]:
s.agg(F.countDistinct('c19')).collect()[0]

Row(count(DISTINCT c19)=3)

In [16]:
for c in s.columns:
    if c == 'label':
        continue
    elif 'n' in c:
        continue
    elif c == 'c8' or c == 'c19':
        continue
    else:
        s = s.drop(c)
s.columns

['label', 'n0', 'n1', 'n2', 'c8', 'c19']

# Train/Test Split

In [22]:
# on sample
trainSample, testSample = s.randomSplit([1.0, 9.0], 666)
trainSample = trainSample.cache()
testSample = testSample.cache()

In [23]:
trainSample.count(), testSample.count()

(1171, 10365)

In [24]:
trainSample.head()

Row(label=0.0, n0=None, n1=-1.0, n2=None, c8='7cc72ec2', c19=None)

In [6]:
# on full
test, train = train.randomSplit([1.0, 9.0])
# train = train.cache()
# test = test.cache()

In [7]:
# %%time
start = time.time()
print(train.count(), test.count())
print((time.time() - start) / 60)

41255346 4585271
21.664877021312712


# Normalize numerical data

In [25]:
# %%time
stats = trainSample[trainSample.columns[1:4]].describe()
maxes = np.array(stats[stats['summary'] == 'max'].collect())[0][1:]
mins = np.array(stats[stats['summary'] == 'min'].collect())[0][1:]
maxes = [float(m) for m in maxes]
mins = [float(m) for m in mins]

CPU times: user 10 ms, sys: 30 ms, total: 40 ms
Wall time: 588 ms


In [26]:
# normalize columns
for i,c in enumerate(trainSample.columns[1:4]):
    trainSample = trainSample.withColumn(c, (trainSample[c] - mins[i]) / (maxes[i] - mins[i]))
    testSample = testSample.withColumn(c, (testSample[c] - mins[i]) / (maxes[i] - mins[i]))

In [27]:
trainSample.head()

Row(label=0.0, n0=None, n1=0.0001595405232929164, n2=None, c8='7cc72ec2', c19=None)

In [28]:
testSample.head()

Row(label=0.0, n0=None, n1=0.0001595405232929164, n2=None, c8='7cc72ec2', c19=None)

In [29]:
# # need to decide on proper NA handling later on, for now filling with 0
trainSample = trainSample.na.fill(0, subset=trainSample.columns[1:4])
testSample = testSample.na.fill(0, subset=testSample.columns[1:4])

In [30]:
trainSample.head()

Row(label=0.0, n0=0.0, n1=0.0001595405232929164, n2=0.0, c8='7cc72ec2', c19=None)

In [31]:
testSample.head()

Row(label=0.0, n0=0.0, n1=0.0001595405232929164, n2=0.0, c8='7cc72ec2', c19=None)

In [8]:
def normalizeNumeric(trainDf, testDf):
    stats = trainDf[trainDf.columns[1:14]].describe()
    maxes = np.array(stats[stats['summary'] == 'max'].collect())[0][1:]
    mins = np.array(stats[stats['summary'] == 'min'].collect())[0][1:]
#     means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]
    maxes = [float(m) for m in maxes]
    mins = [float(m) for m in mins]
#     means = [float(m) for m in means]
    
    for i,c in enumerate(trainDf.columns[1:14]):
        trainDf = trainDf.withColumn(c, (trainDf[c] - mins[i]) / (maxes[i] - mins[i]))
        testDf = testDf.withColumn(c, (testDf[c] - mins[i]) / (maxes[i] - mins[i]))
        
    # NEED TO FIGURE THIS OUT FIRST
    # two options --> fill with 0 or fill with mean
    trainDf = trainDf.na.fill(0, subset=trainDf.columns[1:14])
    testDf = testDf.na.fill(0, subset=testDf.columns[1:14])
    trainDf = trainDf.cache()
    testDf = testDf.cache()
    
    return trainDf, testDf

## on sample

In [ ]:
# on sample
trainSample, testSample = normalizeNumeric(trainSample, testSample)

In [ ]:
trainSample.head()

In [ ]:
testSample.head()

## on full

In [9]:
start = time.time()
train, test = normalizeNumeric(train, test)
print((time.time() - start) / 60)

14.324187703927358


In [10]:
train.head()

Row(label=0.0, n0=0.0, n1=3.880812486902258e-06, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='05db9164', c1='31eb7ac1', c2='2dc2b523', c3='137a5e26', c4='25c83c98', c5='fe6b92e5', c6='ce8217f8', c7='5b392875', c8='7cc72ec2', c9='3b08e48b', c10='9d12ce9b', c11='d9510218', c12='9dfda2b9', c13='07d13a8f', c14='6cb56b0f', c15='57ddd4e0', c16='2005abd1', c17='e3f6ec41', c18=None, c19=None, c20='45458c05', c21=None, c22='be7c41b4', c23='1793a828', c24=None, c25=None)

In [11]:
test.head()

Row(label=0.0, n0=0.0, n1=3.880812486902258e-06, n2=0.0, n3=0.0, n4=0.00012478704162999338, n5=0.0, n6=0.0, n7=0.0014883413262774929, n8=0.0005513629001688549, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='8cf07265', c1='f6f4fe4b', c2='680f077d', c3='e9cac79c', c4='25c83c98', c5='fe6b92e5', c6='2e0b99f0', c7='0b153874', c8='a73ee510', c9='fa6a0a1b', c10='6b5f6a88', c11='7fbb6d63', c12='1399de53', c13='07d13a8f', c14='ef6fe5a5', c15='b8beb278', c16='1e88c74f', c17='0a20b09c', c18=None, c19=None, c20='042304c2', c21=None, c22='32c7478e', c23='59c12dda', c24=None, c25=None)

# Categorical manipulation

In [63]:
# counts = trainSample.select(trainSample.columns[14:]).summary('count')

In [64]:
# c = trainSample.groupBy('c0').count()
# c.collect()[0]
# # type(c)
# # c = c.orderBy(c.count.desc()).collect()

In [65]:
# def findInfrequentValues(c, n=10):
#     # c is the column that we are operating on
#     # 
#     counts = trainSample.groupBy(c).count()
#     infrequentValues = counts.filter(counts['count'] <= n)
#     s = infrequentValues.agg(F.collect_set(c)).collect()[0][0]
#     return s

In [66]:
# names = replaceInfrequentValues('c0')
# type(names)

In [67]:
# df = trainSample.withColumn('c0', F.when(trainSample['c0'] == names[0], '999').otherwise(trainSample['c0']))

In [68]:
# df.select(df['c0'] == names[0]).collect()

In [69]:
# names[0]

In [70]:
# int('0x' + names[0], 16)

In [17]:
# convert hex values to integers and modulo them
def hashValues(row):
    if row != None:
        return str(int('0x' + row, 16) % 10000)
    else:
        return str(row)

In [18]:
udf_object = F.udf(hashValues)

In [ ]:
# trainSample.select('c0').collect()[0]

In [ ]:
# trainSample.withColumn('c0', udf_object(trainSample['c0'])).select('c0').collect()[0]

In [17]:
# for c in trainSample.columns[14:]:
#     trainSample = trainSample.withColumn(c, udf_object(trainSample[c]))
#     testSample = testSample.withColumn(c, udf_object(testSample[c]))

    
# # trainSample = trainSample.cache()

In [ ]:
# trainSample.head()

In [ ]:
# testSample.head()

In [ ]:
# %%time
# for c in trainSample.columns[14:]:
#     newCol = c + '_idx'
#     indexer = StringIndexer(inputCol=c, outputCol=newCol, handleInvalid='keep')
# #     trainSample = indexer.fit(trainSample).transform(trainSample)
#     sIdx = indexer.fit(trainSample)
#     trainSample = sIdx.transform(trainSample)
#     testSample = sIdx.transform(testSample)
# trainSample = trainSample.cache()
# testSample = testSample.cache()

In [ ]:
# trainSample.select(trainSample.columns[40:]).show(2)

In [ ]:
# testSample.select(testSample.columns[40:]).show(2)

In [26]:
# cols = trainSample.columns[40:]
# outputCols = [c.strip('_idx') + '_OHE' for c in cols]
# encoder = OneHotEncoderEstimator(inputCols=cols, outputCols=outputCols)
# OHE = encoder.fit(trainSample)
# trainSample = OHE.transform(trainSample)
# testSample = OHE.transform(testSample)

In [ ]:
# trainSample.select(trainSample.columns[-26:]).show(2)

In [ ]:
# testSample.select(testSample.columns[-26:]).show(2)

In [29]:
# cols = [c for c in trainSample.columns if 'n' in c or 'OHE' in c]

In [30]:
# v = VectorAssembler(inputCols=cols, outputCol="features")
# trainSample = v.transform(trainSample)
# testSample = v.transform(testSample)

In [ ]:
# trainSample.columns[-1], testSample.columns[-1]

In [ ]:
# trainSample.select('features').head()

In [ ]:
# testSample.select('features').head()

In [32]:
len(trainSample.columns)

6

In [19]:
def createFeatureVector(trainDf, testDf):
    # create hash function for binning categorical variables
    def hashValues(row):
        if row != None:
            # return integer value of hex label, modulo by 10000 (keep only the last 4 digits)
            return str(int('0x' + row, 16) % 10000)
        else:
            return str(row)
    
    # create the udf object from the helper function
    udf_object = F.udf(hashValues)
    
    # hash all hex strings in both train and test
    for c in trainDf.columns[14:]:
        trainDf = trainDf.withColumn(c, udf_object(trainDf[c]))
        testDf = testDf.withColumn(c, udf_object(testDf[c]))
        
    # index the hash values into categories
    for c in trainDf.columns[14:]:
        newCol = c + '_idx'
        indexer = StringIndexer(inputCol=c, outputCol=newCol, handleInvalid='keep')
        f = indexer.fit(trainDf)
        trainDf = f.transform(trainDf)
        testDf = f.transform(testDf)
        
    # One-hot encode the categorical indices
    inputCols = trainDf.columns[6:]
    outputCols = [c.strip('_idx') + '_OHE' for c in inputCols]
    # NEED TO DECIDE ON DROPLAST
    encoder = OneHotEncoderEstimator(inputCols=inputCols, outputCols=outputCols)
#     encoder = OneHotEncoderEstimator(inputCols=inputCols, outputCols=outputCols, dropLast=False)
    e = encoder.fit(trainDf)
    trainDf = e.transform(trainDf)
    testDf = e.transform(testDf)
    
    # assemble all features into single SparseVector column
    cols = [c for c in trainDf.columns if 'n' in c or 'OHE' in c]
    v = VectorAssembler(inputCols=cols, outputCol="features")
    trainDf = v.transform(trainDf)
    testDf = v.transform(testDf)
    trainDf = trainDf.cache()
    testDf = testDf.cache()
    
    return trainDf, testDf

## on sample

In [35]:
# on sample
trainSample, testSample = createFeatureVector(trainSample, testSample, 10000)

In [36]:
trainSample.columns[-1], testSample.columns[-1]

('features', 'features')

In [37]:
trainSample.select('features').head()

Row(features=SparseVector(9, {1: 0.0002, 4: 1.0, 5: 1.0}))

In [38]:
testSample.select('features').head()

Row(features=SparseVector(9, {1: 0.0002, 4: 1.0, 5: 1.0}))

# create smaller sample for Divya

In [39]:
# trainSample.write.format('parquet').save('data/trainSample.parquet')
# testSample.write.format('parquet').save('data/testSample.parquet')
trainSample.write.format('parquet').save('data/smallTrainSample.parquet')
testSample.write.format('parquet').save('data/smallTestSample.parquet')

## on full

In [20]:
# %%time
# on full
start = time.time()
train, test = createFeatureVector(train, test)
print((time.time() - start) / 60)

Py4JJavaError: An error occurred while calling o1078.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 13.0 failed 1 times, most recent failure: Lost task 2.0 in stage 13.0 (TID 105, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.sql.execution.columnar.ColumnBuilder$.ensureFreeSpace(ColumnBuilder.scala:157)
	at org.apache.spark.sql.execution.columnar.BasicColumnBuilder.appendFrom(ColumnBuilder.scala:71)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.org$apache$spark$sql$execution$columnar$NullableColumnBuilder$$super$appendFrom(ColumnBuilder.scala:97)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder$class.appendFrom(NullableColumnBuilder.scala:61)
	at org.apache.spark.sql.execution.columnar.NativeColumnBuilder.org$apache$spark$sql$execution$columnar$compre

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 41750)
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
---------------------------------

In [ ]:
train.select('features').head()

In [ ]:
test.select('features').head()

In [ ]:
tpp = train.select('label','features')
dpp = test.select('label', 'features')

In [ ]:
tpp.columns, dpp.columns

In [ ]:
tpp.write.format('parquet').save('data/trainPreProcessed.parquet')
dpp.write.format('parquet').save('data/devPreProcessed.parquet')

# pipeline implementation

In [38]:
def hashCategoricals(trainDf, testDf):
    # create hash function for binning categorical variables
    def hashValues(row):
        if row != None:
            # return integer value of hex label, modulo by 10000 (keep only the last 4 digits)
            return str(int('0x' + row, 16) % 10000)
        else:
            return str(row)
    
    # create the udf object from the helper function
    udf_object = F.udf(hashValues)
    
    # hash all hex strings in both train and test
    for c in trainDf.columns[14:]:
        trainDf = trainDf.withColumn(c, udf_object(trainDf[c]))
        testDf = testDf.withColumn(c, udf_object(testDf[c]))
    
    return trainDf.cache(), testDf.cache()

In [39]:
trainSample, testSample = hashCategoricals(trainSample, testSample)

In [40]:
trainSample.head()

Row(label=0.0, n0=0.0, n1=0.0002068680182043856, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='8852', c1='8017', c2='4310', c3='8313', c4='4343', c5='2821', c6='3090', c7='84', c8='8418', c9='8539', c10='2816', c11='6109', c12='1890', c13='6422', c14='4146', c15='4642', c16='2577', c17='8733', c18='None', c19='None', c20='5227', c21='9419', c22='5356', c23='1146', c24='None', c25='None')

In [41]:
testSample.head()

Row(label=0.0, n0=0.0, n1=0.0002068680182043856, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='5684', c1='8017', c2='9656', c3='9545', c4='9704', c5='8079', c6='3090', c7='84', c8='8418', c9='8539', c10='2816', c11='1488', c12='1890', c13='2527', c14='5555', c15='7942', c16='2577', c17='7231', c18='None', c19='None', c20='1497', c21='None', c22='3739', c23='1592', c24='None', c25='None')

In [52]:
# pipeline implementation
def createFeatureVector2(trainDf, testDf):
    # generate stages for pipeline
    stages = []
    
    # create indexer to hash values into categories
    for c in trainDf.columns[14:]:
        strIdxCol = c + '_idx'
        oheCol = strIdxCol.strip('_idx') + '_OHE'
        indexer = StringIndexer(inputCol=c, outputCol=strIdxCol, handleInvalid='keep')
        OHE = OneHotEncoderEstimator(inputCols=strIdxCol, outputCols=oheCol, dropLast=False)
        stages += [indexer, OHE]
        
    # One-hot encode the categorical indices
#     inputCols = trainDf.columns[40:]
#     outputCols = [c.strip('_idx') + '_OHE' for c in inputCols]
#     encoder = OneHotEncoderEstimator(inputCols=inputCols, outputCols=outputCols, dropLast=False)
#     stages += [encoder]
#     print(stages)
#     e = encoder.fit(trainDf)
#     trainDf = e.transform(trainDf)
#     testDf = e.transform(testDf)
    
    # assemble all features into single SparseVector column
#     cols = [c for c in trainDf.columns if 'n' in c or 'OHE' in c]
#     v = VectorAssembler(inputCols=cols, outputCol="features")
#     stages += [v]
#     trainDf = v.transform(trainDf)
#     testDf = v.transform(testDf)
    
    pipe = Pipeline(stages=stages)
    model = pipe.fit(trainDf)
    trainDf = model.transform(trainDf)
    testDf = model.transform(testDf)
    
    return trainDf.cache(), testDf.cache()

In [54]:
trainSample.head()

Row(label=0.0, n0=0.0, n1=0.0002068680182043856, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='8852', c1='8017', c2='4310', c3='8313', c4='4343', c5='2821', c6='3090', c7='84', c8='8418', c9='8539', c10='2816', c11='6109', c12='1890', c13='6422', c14='4146', c15='4642', c16='2577', c17='8733', c18='None', c19='None', c20='5227', c21='9419', c22='5356', c23='1146', c24='None', c25='None')

In [ ]:
trainer, tester = createFeatureVector2(trainSample, testSample)

In [24]:
trainer.columns[-1], tester.columns[-1]

('features', 'features')

In [48]:
trainer.head()

Row(label=0.0, n0=0.0, n1=0.0002068680182043856, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='8852', c1='8017', c2='4310', c3='8313', c4='4343', c5='2821', c6='3090', c7='84', c8='8418', c9='8539', c10='2816', c11='6109', c12='1890', c13='6422', c14='4146', c15='4642', c16='2577', c17='8733', c18='None', c19='None', c20='5227', c21='9419', c22='5356', c23='1146', c24='None', c25='None', c0_idx=1.0, c1_idx=0.0, c2_idx=2385.0, c3_idx=1150.0, c4_idx=6.0, c5_idx=2.0, c6_idx=68.0, c7_idx=0.0, c8_idx=1.0, c9_idx=0.0, c10_idx=97.0, c11_idx=231.0, c12_idx=12.0, c13_idx=0.0, c14_idx=969.0, c15_idx=1788.0, c16_idx=8.0, c17_idx=811.0, c18_idx=0.0, c19_idx=0.0, c20_idx=2235.0, c21_idx=1.0, c22_idx=3.0, c23_idx=361.0, c24_idx=0.0, c25_idx=0.0)

In [49]:
tester.head()

Row(label=0.0, n0=0.0, n1=0.0002068680182043856, n2=0.0, n3=0.0, n4=0.0, n5=0.0, n6=0.0, n7=0.0, n8=0.0, n9=0.0, n10=0.0, n11=0.0, n12=0.0, c0='5684', c1='8017', c2='9656', c3='9545', c4='9704', c5='8079', c6='3090', c7='84', c8='8418', c9='8539', c10='2816', c11='1488', c12='1890', c13='2527', c14='5555', c15='7942', c16='2577', c17='7231', c18='None', c19='None', c20='1497', c21='None', c22='3739', c23='1592', c24='None', c25='None', c0_idx=0.0, c1_idx=0.0, c2_idx=140.0, c3_idx=7.0, c4_idx=0.0, c5_idx=0.0, c6_idx=68.0, c7_idx=0.0, c8_idx=1.0, c9_idx=0.0, c10_idx=97.0, c11_idx=120.0, c12_idx=12.0, c13_idx=1.0, c14_idx=149.0, c15_idx=7.0, c16_idx=8.0, c17_idx=43.0, c18_idx=0.0, c19_idx=0.0, c20_idx=7.0, c21_idx=0.0, c22_idx=1.0, c23_idx=1.0, c24_idx=0.0, c25_idx=0.0)

In [86]:
trainSample.write.format('parquet').save('data/trainSample.parquet')